In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2 # OpenCV for image processing

print("TensorFlow Version:", tf.__version__)


In [ ]:
BASE_INPUT_PATH = '/kaggle/input/downsized-data'
MODEL_PATH = '/kaggle/input/nih-phase-4-further-fine-tuning/best_further_fine_tuned_model.keras'
IMAGE_DIR = os.path.join(BASE_INPUT_PATH, 'nih_images_resized_256/')
CSV_PATH = os.path.join(BASE_INPUT_PATH, 'Data_Entry_2017_stratified_downsized.csv')


In [ ]:
IMG_SIZE=224
BATCH_SIZE=32
AUTOTUNE= tf.data.AUTOTUNE

In [ ]:
FINAL_14_LABELS = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
    'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule',
    'Pneumonia', 'Pneumothorax', 'No Finding'
]

In [ ]:
print("--- Preparing Validation Data ---")
try:
    df = pd.read_csv(CSV_PATH)

    # One-hot encode the labels
    for label in FINAL_14_LABELS:
        df[label] = df['Finding Labels'].str.lower().apply(
            lambda finding_string: 1 if label.lower() in str(finding_string) else 0
        )
    df['image_path'] = df['Image Index'].apply(lambda x: os.path.join(IMAGE_DIR, x))
    _,val_df = train_test_split(df, test_size=0.15, random_state=42)

    print(f"Validation set size: {len(val_df)} images")

    # --- TensorFlow Dataset Pipeline ---
    def parse_function(image_path, labels):
        """Loads and preprocesses an image."""
        img_raw = tf.io.read_file(image_path)
        img = tf.io.decode_png(img_raw, channels=1)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.grayscale_to_rgb(img) # Model expects 3 channels
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        return img, labels

    def create_dataset(dataframe):
        """Creates a tf.data.Dataset from a pandas DataFrame."""
        image_paths = dataframe['image_path'].values
        labels = dataframe[FINAL_14_LABELS].values.astype(np.float32)
        ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
        ds = ds.map(parse_function, num_parallel_calls=AUTOTUNE)
        return ds

    # Create the validation dataset
    validation_dataset = (
        create_dataset(val_df)
        .batch(BATCH_SIZE)
        .prefetch(buffer_size=AUTOTUNE)
    )

    print("\nComplete: `validation_dataset` is ready.")
    
    # Clean up to free memory
    del df
    gc.collect()

except FileNotFoundError:
    print("Error: Data files not found. Please ensure the CSV and image directories are correctly specified.")
    validation_dataset = None


In [ ]:
print(f"--- Loading Model from: {MODEL_PATH} ---")
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    model.summary()
    print("\nModel loaded successfully.")
except (IOError, FileNotFoundError) as e:
    print(f"Error loading model: {e}")
    print("Please ensure the model path is correct and the file is available.")
    model = None


In [ ]:
if model and validation_dataset:
    print("--- Evaluating Model Performance on Validation Data ---")
    
    y_true = np.concatenate([y for x, y in validation_dataset], axis=0)
    print("Generating predictions...")
    y_pred_probs = model.predict(validation_dataset, verbose=1)
    y_pred_binary = (y_pred_probs > 0.5).astype(int)

    # --- Classification Report ---
    # This report shows precision, recall, and F1-score for each class.
    print("\n\n--- Classification Report ---")
    report = classification_report(y_true, y_pred_binary, target_names=FINAL_14_LABELS)
    print(report)

    # --- Confusion Matrices ---
    # For multi-label classification
    print("\n--- Plotting Confusion Matrices for Each Label ---")
    mcm = multilabel_confusion_matrix(y_true, y_pred_binary)
    
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    
    # Remove the last two subplots as we only have 14 labels
    fig.delaxes(axes[14])
    fig.delaxes(axes[15])

    for i, (matrix, label) in enumerate(zip(mcm, FINAL_14_LABELS)):
        sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                    xticklabels=['Predicted Negative', 'Predicted Positive'],
                    yticklabels=['Actual Negative', 'Actual Positive'],
                    cbar=False)
        axes[i].set_title(f'Confusion Matrix: {label}', fontsize=12)
    
    plt.suptitle('Confusion Matrix for Each Pathology (Validation Set)', fontsize=20, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("Model or validation data not available. Skipping evaluation.")

